# Exercise Set 8: Introduction to Web Scraping

*Afternoon, August 16, 2018*

In this Exercise Set we shall practice our webscraping skills utiilizing only basic python. We shall cover variations between static and dynamic pages. 

## Exercise Section 8.1: Scraping Jobnet.dk

This exercise you get to practice locating the request that the JavaScript sends to get the job data that it builds the joblistings from. You should use the **>Network Monitor<** tool in your browser.

Furthermore you practice spotting how the pagination is done, without clicking on the next page button, but instead changing a small parameter in the URL.

> **Ex. 8.1.1:** Hit the joblisting webpage here: https://job.jobnet.dk/CV and locate the request that gets the joblisting data using the a tool called the **>Network Monitor<**. To do this open monitor tool and press the search button on the website. Go to the _network_ pane in the monitor and look at the results. Which two _methods_ does your browser use to communicate with the webserver? What does status code 200 mean?
>
>> _Hint:_ The network monitor is launched by pressing ctrl+shift+i in most browsers. Filter by XHR files.

In [ ]:
# [Answer to Ex. 8.1.1 here]

# The answer is GET and POST requests. Status code 200 means a request was successful.

> **Ex. 8.1.2.:** Use the `request` module to collect the first page of job postings and unpack the relevant `json` data into a `pandas` DataFrame.

In [6]:
# [Answer to Ex. 8.1.2 here]

# This will be in assignment 1

> **Ex. 8.1.3.:** Store and print the 'TotalResultCount' value for later use. Also create a dataframe from the 'JobPositionPostings' field in the json. 

In [5]:
# [Answer to Ex. 8.1.3 here]

# This will be in assignment 1

> **Ex. 8.1.4:** This exercise is about paging the results. We need to understand the websites pagination scheme. Scroll down the webpage and press the next page button. Describe how the parameters of the url changes as you turn the pages.


In [ ]:
# [Answer to Ex. 8.1.4 here]

# The url has a parameter called 'offset', the page yields 20 postings ofset some value from the best matching add. 

> **Ex. 8.1.5:** Design a`for` loop using the `range` function that changes this paging parameter in the URL. Use the TotalResultCount parameter from before to define the limits of the range function. Store these urls in a container. 
>
>**Bonus** Change the SortValue parameter from BestMatch to CreationDate, to make the sorting amendable to updating results daily.
>
>> _Hint:_ See that the parameter is an offset and that this relates to the number of results pr. call made.

In [4]:
# [Answer to Ex. 8.1.5 here]

links = []

for offset in range(0,n_listings+20,20):
    url = 'https://job.jobnet.dk/CV/FindWork/Search?SortValue=CreationDate&Offset={o}'.format(o = offset)
    links.append(url)

> **Ex. 8.1.6:** Pick 20 random links using the `choice` function in the `random` module;  collect the links using the `request` module. Also use the `time.sleep()` function to limit the rate of your calls. Make sure to save the links already collected in a `set()` container to avoid having to reload links already collected. ***extra***: monitor the time left to completing the loop by using `tqdm.tqdm()` function.

In [7]:
# [Answer to Ex. 8.1.6 here]

import random
import time
import tqdm

done = set()
data = []

random_links = random.sample(links,20)

for url in tqdm.tqdm(random_links):
    response = requests.get(url)
    
    if response.ok:
        d = response.json()
    else:
        print('error')

    data += d['JobPositionPostings']
    time.sleep(0.5)

100%|██████████| 20/20 [02:00<00:00,  6.04s/it]


> **Ex. 8.1.7:** Load all the results into a DataFrame.

In [8]:
# [Answer to Ex. 8.1.7 here]

df = pd.DataFrame(data)
df.sample(5)

,AnonymousEmployer,AssignmentStartDate,AutomatchType,Country,EmploymentType,HasLocationValues,HiringOrgCVR,HiringOrgName,ID,IsExternal,...,UseWorkPlaceAddressForJoblog,Weight,WorkHours,WorkPlaceAbroad,WorkPlaceAddress,WorkPlaceCity,WorkPlaceNotStatic,WorkPlaceOtherAddress,WorkPlacePostalCode,WorkplaceID
343,False,2018-09-10T12:49:44.2365976+02:00,0,Danmark,Fastansættelse,False,0,Bureau søger for kunde,E6658473,True,...,True,1.0,Fuldtid,False,,Gråsten,False,False,6300,0
367,False,2018-08-23T00:00:00,0,Danmark,Fastansættelse,False,0,Carlsberg Breweries A/S,E6745156,True,...,True,1.0,Fuldtid,False,,Aalborg,False,False,9000,0
158,False,0001-01-01T00:00:00,0,Danmark,,True,13760381,VAM A/S,4857177,False,...,False,1.0,Fuldtid,False,,,False,True,,77552
389,False,0001-01-01T00:00:00,0,Danmark,,True,27215556,Dynaudio A/S,4862289,False,...,True,1.0,Fuldtid,False,Sverigesvej 15,Skanderborg,False,False,8660,16566
305,False,0001-01-01T00:00:00,0,Danmark,,True,30487753,ENTREPRENØR JENS KARSTENSEN A/S,4845632,False,...,True,1.0,Fuldtid,False,Torstedalle 1,Horsens,False,False,8700,127628


## Exercise Section 8.2: Scraping Trustpilot.com
Now for a slightly more elaborate, yet still simple scraping problem. Here we want to scrape trustpilot for user reviews. This data is very nice since it provides free labeled data (rating) to train a machine learning model to understand positive and negative sentiment. 

Here you will practice crawling a website collecting the links to each company review page, and finally locate another behind the scenes JavaScript request that gets the review data in a neat json format.

> **Ex. 8.2.1:** Visit the https://www.trustpilot.com/ website and locate the categories page. From this page you find links to company listings. Get the category page using the `requests` module and extract each link to a specific category page from the HTML. This can be done using the basic python `.split()` string method. Make sure only links within the ***/categories/*** section are kept, checking each string using the ```if 'pattern' in string``` condition. 
>
>> *Hint:* The links are relative. You need to add the domain name
>>
>> *Hint #2:* You will need to convert the page response to a string, using the `.text` property.


In [16]:
# [Answer to Ex. 8.2.1]

url = 'https://www.trustpilot.com/categories/'
response = requests.get(url)

if response.ok:
    html = response.text
else:
    print('error')

links = set()

for link_loc in html.split('href="')[1:]:
    link = link_loc.split('"')[0]
    
    if '/categories/' in link:
        links.add(link)

print(f"there are {str(len(links))} links, the first one is {list(links)[0]}") # link is relative

links = ['https://www.trustpilot.com'+link for link in links]# add the domain to each link

there are 28, the first one is /categories/home_garden


['https://www.trustpilot.com/categories/home_garden',
 'https://www.trustpilot.com/categories/utilities',
 'https://www.trustpilot.com/categories/tobacco-products',
 'https://www.trustpilot.com/categories/art',
 'https://www.trustpilot.com/categories/mobile_internet',
 'https://www.trustpilot.com/categories/travel_holidays',
 'https://www.trustpilot.com/categories/transportation',
 'https://www.trustpilot.com/categories/leisure',
 'https://www.trustpilot.com/categories/craftsman',
 'https://www.trustpilot.com/categories/electronics',
 'https://www.trustpilot.com/categories/sport',
 'https://www.trustpilot.com/categories/money',
 'https://www.trustpilot.com/categories/health_wellbeing',
 'https://www.trustpilot.com/categories/kids',
 'https://www.trustpilot.com/categories/animals_and_pets',
 'https://www.trustpilot.com/categories/other',
 'https://www.trustpilot.com/categories/food_beverage',
 'https://www.trustpilot.com/categories/cloud_computing',
 'https://www.trustpilot.com/categori

> **Ex. 8.2.2:** Get one of the category section links (any one will do). Write a function to extract the links to the company review pages from the HTML.


In [20]:
# [Answer to Ex. 8.2.2]

# Getting the page 
url = 'https://www.trustpilot.com/categories/art'

response = requests.get(url)
if response.ok:
    html = response.text
else:
    print('error')

# get links function
def get_links(html):
    links = set() # define container    
    for link_loc in html.split('href="')[1:]: # locate the start of a link
        link = link_loc.split('"')[0] # split at the end of the link
        links.add(link) # if it is: add it to the set container
    return links

# A function to extract links to company review pages
def get_company_links(html):
    return [link for link in get_links(html) if '/review/' in link] # check if the /review/ pattern is in the link and keep if true


company_links = get_company_links(html)
print(len(company_links))

20


> **Ex. 8.2.3:** Figure out how the pagination is done, by following how the url changes when pressing the **next page**-button to obtain more company listings. Write a function that builds links to paging all the company listing results of each category. This includes parsing the number of subpages of each category and changing the correct parameter in the url.
>
>> *Hint:* Find the maximum number of result pages, right before the next page button and make a loop change the page parameter of the url.


In [23]:
# [Answer to Ex.8.2.3]

def get_all_category_pages(category_link): 
    # Get the broad category html
    response = requests.get(category_link)
    if response.ok:
        html = response.text
    else:
        print('error')
        return False
    
    # Get the links in this html
    links = get_links(html)

    # find the max_page number.
    page_links = [link for link in links if '?page=' in link] # check if the paging parameter is in the link
    if len(page_links)==0: # no pages.
        return [category_link]
    n_pages = max([int(link.split('page=')[-1]) for link in page_links]) # extract the page value and take the max
    
    paging_links = [category_link] # define container and store the original result page
    
    q = category_link+'?page=%d' # define the varying parameter string.

    for num in range(2,n_pages+1): # build the links.
        paging_links.append(q%num)
    
    return paging_links

pages = get_all_category_pages(url)
pages

['https://www.trustpilot.com/categories/art',
 'https://www.trustpilot.com/categories/art?page=2',
 'https://www.trustpilot.com/categories/art?page=3',
 'https://www.trustpilot.com/categories/art?page=4',
 'https://www.trustpilot.com/categories/art?page=5',
 'https://www.trustpilot.com/categories/art?page=6',
 'https://www.trustpilot.com/categories/art?page=7',
 'https://www.trustpilot.com/categories/art?page=8']

> **Ex. 8.2.4:** Loop through all categories and build the paging links using the above defined function.

In [25]:
# [Answer to Ex.8.2.4]

# Build the paging links
company_listings = []

for link in links: 
    if 'support.trustpilot' in link:
        continue
    company_listings+=get_all_category_pages(link)
    print(link)
    
print('We need to visit %d company listing pages to collect all company addresses'%len(company_listings))

https://www.trustpilot.com/categories/home_garden
https://www.trustpilot.com/categories/utilities
https://www.trustpilot.com/categories/tobacco-products
https://www.trustpilot.com/categories/art
https://www.trustpilot.com/categories/mobile_internet
https://www.trustpilot.com/categories/travel_holidays
https://www.trustpilot.com/categories/transportation
https://www.trustpilot.com/categories/leisure
https://www.trustpilot.com/categories/craftsman
https://www.trustpilot.com/categories/electronics
https://www.trustpilot.com/categories/sport
https://www.trustpilot.com/categories/money
https://www.trustpilot.com/categories/health_wellbeing
https://www.trustpilot.com/categories/kids
https://www.trustpilot.com/categories/animals_and_pets
https://www.trustpilot.com/categories/other
https://www.trustpilot.com/categories/food_beverage
https://www.trustpilot.com/categories/cloud_computing
https://www.trustpilot.com/categories/gambling
https://www.trustpilot.com/categories/erotic
https://www.trust

> **Ex. 8.2.5:** Randomly pick one of category listing links you have generated, and get the links to the companies listed using the other function defined. 


In [38]:
# [Answer to Ex.8.2.5]

company_page_link = random.choice(company_listings)

response = requests.get(company_page_link)
if response.ok:
    html = response.text
else:
    print('error')

company_links = ['https://www.trustpilot.com' + link for link in get_company_links(html)] # use the above defined function
company_links

['https://www.trustpilot.com/review/uniwigs.com',
 'https://www.trustpilot.com/review/mininthebox.com',
 'https://www.trustpilot.com/review/www.modli.co',
 'https://www.trustpilot.com/review/yessstyle.com',
 'https://www.trustpilot.com/review/www.nikestore.com',
 'https://www.trustpilot.com/review/patches4pennies.com',
 'https://www.trustpilot.com/review/hair-professionnel.com',
 'https://www.trustpilot.com/review/www.go2dressu.com',
 'https://www.trustpilot.com/review/www.unitedshades.com',
 'https://www.trustpilot.com/review/www.pooz.com',
 'https://www.trustpilot.com/review/www.fredflare.com',
 'https://www.trustpilot.com/review/diamondveneer.com',
 'https://www.trustpilot.com/review/paom.com',
 'https://www.trustpilot.com/review/onecklace.com',
 'https://www.trustpilot.com/review/coedstore.co',
 'https://www.trustpilot.com/review/www.princetonwatches.com',
 'https://www.trustpilot.com/review/replacementlensexpress.com',
 'https://www.trustpilot.com/review/onlybandanas.com',
 'https

> **Ex. 8.2.6:** Visit one of these links and inspect the **>Network Monitor<** to locate the request that loads the review data. Use the requests module to retrieve this link and unpack the json results to a pandas DataFrame.
>
>> _Hint:_ Look for a request which sends a the link to a file called `jsonId`

In [39]:
# [Answer to Ex.8.2.6]

# This link will be unique to the company you choose from the above list.
direct_link = 'https://www.trustpilot.com/review/59f33de00000ff0005aed171/jsonld' 

response = requests.get(direct_link)
d = response.json() # parse json using the build-in function.
pd.DataFrame(d['review'])

,@type,author,datePublished,headline,inLanguage,itemReviewed,publisher,reviewBody,reviewRating
0,Review,"{'@type': 'Person', 'name': 'Eugene', 'url': '...",2018-08-09T00:32:26Z,Logo Orbit exceeded my expectations in…,en,"{'@type': 'Thing', 'name': 'Logo Orbit'}","{'@type': 'Organization', 'name': 'Trustpilot'...",Logo Orbit exceeded my expectations in every w...,"{'@type': 'Rating', 'bestRating': '5', 'worstR..."
1,Review,"{'@type': 'Person', 'name': 'Abraham Anguiano ...",2018-08-08T20:47:20Z,They work with you on every detail !,en,"{'@type': 'Thing', 'name': 'Logo Orbit'}","{'@type': 'Organization', 'name': 'Trustpilot'...",They work with you on every detail ! I bought ...,"{'@type': 'Rating', 'bestRating': '5', 'worstR..."
2,Review,"{'@type': 'Person', 'name': 'Kathryn Pierce Fo...",2018-08-08T19:32:28Z,Perfect for Campus Ministry,en,"{'@type': 'Thing', 'name': 'Logo Orbit'}","{'@type': 'Organization', 'name': 'Trustpilot'...",I needed to do a few logs for my campus minist...,"{'@type': 'Rating', 'bestRating': '5', 'worstR..."
3,Review,"{'@type': 'Person', 'name': 'Tonya Cross', 'ur...",2018-08-01T19:31:22Z,Excellent and very professional…,en,"{'@type': 'Thing', 'name': 'Logo Orbit'}","{'@type': 'Organization', 'name': 'Trustpilot'...",Excellent and very professional service. I'm j...,"{'@type': 'Rating', 'bestRating': '5', 'worstR..."
4,Review,"{'@type': 'Person', 'name': 'Remi Valdez', 'ur...",2018-07-24T21:26:33Z,Exceptional customer service.,en,"{'@type': 'Thing', 'name': 'Logo Orbit'}","{'@type': 'Organization', 'name': 'Trustpilot'...","Starting working with David, who helped me get...","{'@type': 'Rating', 'bestRating': '5', 'worstR..."
5,Review,"{'@type': 'Person', 'name': 'MN', 'url': 'http...",2018-07-24T11:47:52Z,Logo Orbit : Your one stop shop for client foc...,en,"{'@type': 'Thing', 'name': 'Logo Orbit'}","{'@type': 'Organization', 'name': 'Trustpilot'...",Logo Orbit is a great place to have your logo ...,"{'@type': 'Rating', 'bestRating': '5', 'worstR..."
6,Review,"{'@type': 'Person', 'name': 'Tangela Mastin', ...",2018-07-20T20:23:37Z,Logo for my travel business,en,"{'@type': 'Thing', 'name': 'Logo Orbit'}","{'@type': 'Organization', 'name': 'Trustpilot'...","Great service,fast,and delivered when they sai...","{'@type': 'Rating', 'bestRating': '5', 'worstR..."
7,Review,"{'@type': 'Person', 'name': 'Peace Of Mind New...",2018-07-10T15:26:39Z,Excellent service,en,"{'@type': 'Thing', 'name': 'Logo Orbit'}","{'@type': 'Organization', 'name': 'Trustpilot'...",Review for LogoOrbit \n\n\nI just started a Ne...,"{'@type': 'Rating', 'bestRating': '5', 'worstR..."
8,Review,"{'@type': 'Person', 'name': 'R&R Party Rental'...",2018-06-13T19:29:09Z,R&R Party Rental,en,"{'@type': 'Thing', 'name': 'Logo Orbit'}","{'@type': 'Organization', 'name': 'Trustpilot'...","Happy Wednesday everyone. First of all, I want...","{'@type': 'Rating', 'bestRating': '5', 'worstR..."
9,Review,"{'@type': 'Person', 'name': 'Leeanna Wynn', 'u...",2018-06-09T02:02:30Z,Logo Orbit has a 5 🌟 customer satisfaction rating,en,"{'@type': 'Thing', 'name': 'Logo Orbit'}","{'@type': 'Organization', 'name': 'Trustpilot'...",Logo Orbit did a great job with our logos. Sam...,"{'@type': 'Rating', 'bestRating': '5', 'worstR..."


Congratulations on coming this far. By now you are almost - still need to figure out how to page the reviews and to find the company ID in the html -, ready to deploy a scraper collecting all reviews on trustpilot. 
If you wanna see just how valuable such data could be visit the follow blogpost: https://blog.openai.com/unsupervised-sentiment-neuron/